In [1]:
import pandas as pd
import numpy as np

from env import get_db_url


## 1. Acquire data from the cloud database.

In [2]:
def get_zillow_data():
    '''
    Function will try to return ad database from csv file if file is local and in same directory.
    IF file doesn't exist it will create and store in same directory
    Otherwise will pull from codeup database.
    Must have credentials for codeup database.
    '''
    query = '''
    SELECT
    prop.*,
    predictions_2017.logerror,
    predictions_2017.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
    FROM properties_2017 prop
    JOIN (
    SELECT parcelid, MAX(transactiondate) AS max_transactiondate
    FROM predictions_2017
    GROUP BY parcelid
    ) pred USING(parcelid)
    JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                      AND pred.max_transactiondate = predictions_2017.transactiondate
    LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
    LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
    LEFT JOIN storytype story USING (storytypeid)
    LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
    WHERE prop.latitude IS NOT NULL
      AND prop.longitude IS NOT NULL
      AND transactiondate <= '2017-12-31'
    '''
    try:
        csv_info = pd.read_csv('zillow.csv', index_col=0 )
        return csv_info
    except FileNotFoundError:
        url = get_db_url('zillow')
        info = pd.read_sql(query, url)
        info.to_csv("zillow.csv", index=True)
        return info

In [3]:
df = get_zillow_data()


/var/folders/ng/82m_197x6jd7ysrq5l2vxd280000gn/T/ipykernel_5221/2524721412.py:40: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_info = pd.read_csv('zillow.csv', index_col=0 )


In [4]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [5]:
df[df.duplicated()]

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc


In [6]:
df.shape

(77380, 68)

In [7]:
df[df.parcelid.duplicated()]


,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc


In [8]:
df.transactiondate = df.transactiondate.str.replace('-', '').astype(int)

In [9]:
df.groupby('parcelid')['transactiondate'].max()

parcelid
10711855     20170707
10711877     20170829
10711888     20170404
10711910     20170317
10711923     20170324
               ...   
167686999    20170228
167687739    20170303
167687839    20170531
167688532    20170203
167689317    20170314
Name: transactiondate, Length: 77380, dtype: int64

In [10]:
df.parcelid.nunique()


77380

In [11]:
df[df.parcelid.duplicated()]

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc


In [12]:
df[df.parcelid == 11393337]

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
116,2463969,11393337,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,6.037235e+13,0.086137,20170608,NaN,NaN,NaN,NaN,"Triplex (3 Units, Any Combination)",NaN,NaN


In [13]:
df.drop_duplicates?

Signature:
df.drop_duplicates(
    subset: 'Hashable | Sequence[Hashable] | None' = None,
    keep: "Literal['first'] | Literal['last'] | Literal[False]" = 'first',
    inplace: 'bool' = False,
    ignore_index: 'bool' = False,
) -> 'DataFrame | None'
Docstring:
Return DataFrame with duplicate rows removed.

Considering certain columns is optional. Indexes, including time indexes
are ignored.

Parameters
----------
subset : column label or sequence of labels, optional
    Only consider certain columns for identifying duplicates, by
    default use all of the columns.
keep : {'first', 'last', False}, default 'first'
    Determines which duplicates (if any) to keep.
    - ``first`` : Drop duplicates except for the first occurrence.
    - ``last`` : Drop duplicates except for the last occurrence.
    - False : Drop all duplicates.
inplace : bool, default False
    Whether to drop duplicates in place or to return a copy.
ignore_index : bool, default False
    If True, the resulting axis wi

In [14]:
df.sort_values('transactiondate', ascending=False, inplace=True)

In [15]:
df.drop_duplicates(subset=['parcelid'], keep='first', inplace=True)

In [16]:
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
77379,1187175,12826780,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,6.037503e+13,0.007204,20170925,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77378,1843709,12773139,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,6.037434e+13,0.037129,20170921,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
77377,2968375,17239384,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,6.111008e+13,0.013209,20170921,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
77376,673515,11000655,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,6.037101e+13,0.020615,20170920,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77375,2864704,10833991,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037132e+13,-0.002245,20170920,Central,NaN,NaN,Central,Condominium,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1447245,17143294,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.111005e+13,-0.020526,20170101,NaN,NaN,NaN,NaN,Condominium,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,20170101,Central,NaN,NaN,Central,Condominium,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,20170101,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,20170101,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN


In [17]:
df[df.parcelid == 11393337]

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
116,2463969,11393337,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,6.037235e+13,0.086137,20170608,NaN,NaN,NaN,NaN,"Triplex (3 Units, Any Combination)",NaN,NaN


In [18]:
df.latitude.isnull().value_counts()

False    77380
Name: latitude, dtype: int64

In [19]:
df = df[~df.latitude.isna()]

In [20]:
df.latitude

77379    33937685.0
77378    34040895.0
77377    34300140.0
77376    34245368.0
77375    34202400.0
            ...    
5        34230044.0
4        34185120.0
3        34245180.0
2        33886168.0
0        33634931.0
Name: latitude, Length: 77380, dtype: float64

In [21]:
df.longitude.isnull().value_counts()

False    77380
Name: longitude, dtype: int64

## 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [22]:
def nulls_by_col(df):
    '''
    This function takes in a dataframe 
    and finds the number of missing values
    it returns a new dataframe with quantity and percent of missing values
    '''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    percent_missing = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [23]:
def nulls_by_row(df):
    '''
    This function takes in a dataframe 
    and finds the number of missing values in a row
    it returns a new dataframe with quantity and percent of missing values
    '''
    num_missing = df.isnull().sum(axis=1)
    percent_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': percent_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [24]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [25]:
summarize(df)

SUMMARY REPORT


Dataframe head: 
            id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
77379  1187175  12826780                    NaN                       NaN   
77378  1843709  12773139                    1.0                       NaN   
77377  2968375  17239384                    NaN                       NaN   

       basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
77379           NaN          2.0         3.0                  NaN   
77378           NaN          1.0         3.0                  NaN   
77377           NaN          2.0         4.0                  NaN   

       buildingqualitytypeid  calculatedbathnbr  ...  censustractandblock  \
77379                    6.0                2.0  ...         6.037503e+13   
77378                    4.0                1.0  ...         6.037434e+13   
77377                    NaN                2.0  ...         6.111008e+13   

       logerror  transactiondate  airconditioningdesc  architecturalsty

In [26]:
def missing_values(df):
    num_rows_missing = df.isnull().sum()
    percent_num_rows_missing = round(num_rows_missing / len(df) * 100, 2)
    return pd.DataFrame({'num_rows_missing': num_rows_missing, 'percent_num_rows_missing': percent_num_rows_missing})


In [27]:
#pd.options.display.max_rows = None

In [28]:
missing_values(df)

,num_rows_missing,percent_num_rows_missing
id,0,0.00
parcelid,0,0.00
airconditioningtypeid,52427,67.75
architecturalstyletypeid,77174,99.73
basementsqft,77330,99.94
...,...,...
buildingclassdesc,77365,99.98
heatingorsystemdesc,27941,36.11
propertylandusedesc,0,0.00
storydesc,77330,99.94


# Prepare

### Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77380 entries, 77379 to 0
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

In [30]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52319
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [31]:
df = df[df.propertylandusedesc == 'Single Family Residential']

In [32]:
def drop_nulls(df, percent):
    '''
    Takes in a dataframe and a percent cutoff to return a dataframe with all the columns that are within the cutoff percentage.

    INPUT:
    df = pandas dataframe
    percent = Null percent cutoff. (0.00)

    OUTPUT:
    new_df = pandas dataframe with all columns that are within the cutoff percentage.
    '''
    original_cols = df.columns.to_list()
    drop_cols = []
    for col in original_cols:
        null_pct = df[col].isna().sum() / df.shape[0]
        if null_pct > percent:
            drop_cols.append(col)
    new_df = df.drop(columns=drop_cols)
    return new_df

In [33]:
df = drop_nulls(df, 0.25)

In [34]:
df.isna().sum() / df.shape[0]

id                              0.000000
parcelid                        0.000000
bathroomcnt                     0.000000
bedroomcnt                      0.000000
calculatedbathnbr               0.002580
calculatedfinishedsquarefeet    0.001548
finishedsquarefeet12            0.004702
fips                            0.000000
fullbathcnt                     0.002580
latitude                        0.000000
longitude                       0.000000
lotsizesquarefeet               0.006996
propertycountylandusecode       0.000000
propertylandusetypeid           0.000000
rawcensustractandblock          0.000000
regionidcity                    0.019802
regionidcounty                  0.000000
regionidzip                     0.000497
roomcnt                         0.000000
yearbuilt                       0.002179
structuretaxvaluedollarcnt      0.001567
taxvaluedollarcnt               0.000019
assessmentyear                  0.000000
landtaxvaluedollarcnt           0.000019
taxamount       

In [35]:
df

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,propertylandusedesc
77379,1187175,12826780,2.0,3.0,2.0,1762.0,1762.0,6037.0,2.0,33937685.0,...,1955.0,140000.0,522000.0,2016.0,382000.0,6317.15,6.037503e+13,0.007204,20170925,Single Family Residential
77378,1843709,12773139,1.0,3.0,1.0,1032.0,1032.0,6037.0,1.0,34040895.0,...,1954.0,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,20170921,Single Family Residential
77377,2968375,17239384,2.0,4.0,2.0,1612.0,1612.0,6111.0,2.0,34300140.0,...,1964.0,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,20170921,Single Family Residential
77376,673515,11000655,2.0,2.0,2.0,1286.0,1286.0,6037.0,2.0,34245368.0,...,1940.0,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,20170920,Single Family Residential
77195,1017567,11388980,2.0,3.0,2.0,1518.0,1518.0,6037.0,2.0,33983643.0,...,1948.0,116897.0,229242.0,2016.0,112345.0,3277.29,6.037703e+13,0.023168,20170919,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,870991,12069064,1.0,2.0,1.0,738.0,738.0,6037.0,1.0,34149214.0,...,1922.0,18890.0,218552.0,2016.0,199662.0,2366.08,6.037302e+13,0.101723,20170101,Single Family Residential
6,781532,12095076,3.0,4.0,3.0,2962.0,2962.0,6037.0,3.0,34145202.0,...,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,20170101,Single Family Residential
3,2288172,12177905,3.0,4.0,3.0,2376.0,2376.0,6037.0,3.0,34245180.0,...,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,20170101,Single Family Residential
2,11677,14186244,2.0,3.0,2.0,1243.0,1243.0,6059.0,2.0,33886168.0,...,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,20170101,Single Family Residential


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52319 entries, 77379 to 0
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52319 non-null  int64  
 1   parcelid                      52319 non-null  int64  
 2   bathroomcnt                   52319 non-null  float64
 3   bedroomcnt                    52319 non-null  float64
 4   calculatedbathnbr             52184 non-null  float64
 5   calculatedfinishedsquarefeet  52238 non-null  float64
 6   finishedsquarefeet12          52073 non-null  float64
 7   fips                          52319 non-null  float64
 8   fullbathcnt                   52184 non-null  float64
 9   latitude                      52319 non-null  float64
 10  longitude                     52319 non-null  float64
 11  lotsizesquarefeet             51953 non-null  float64
 12  propertycountylandusecode     52319 non-null  object 
 13  p

In [37]:
df.columns.to_list()

['id',
 'parcelid',
 'bathroomcnt',
 'bedroomcnt',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fips',
 'fullbathcnt',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidzip',
 'roomcnt',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'assessmentyear',
 'landtaxvaluedollarcnt',
 'taxamount',
 'censustractandblock',
 'logerror',
 'transactiondate',
 'propertylandusedesc']

In [38]:
cols = ['id', 
 'parcelid', 
 'propertylandusetypeid',  
 'propertylandusetypeid', 
 'rawcensustractandblock', 
 'regionidcity',
  'regionidcounty',
 'regionidzip',
 'fips',
 'censustractandblock',

]

In [39]:
cols2 = df.select_dtypes(exclude='object').drop(columns=cols).columns.to_list()
cols2

['bathroomcnt',
 'bedroomcnt',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fullbathcnt',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'roomcnt',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'assessmentyear',
 'landtaxvaluedollarcnt',
 'taxamount',
 'logerror',
 'transactiondate']

In [40]:
def remove_outliers(df, col_list, k=1.5):
    '''
    remove outliers from a dataframe based on a list of columns
    using the tukey method.
    returns a single dataframe with outliers removed
    '''
    col_qs = {}
    for col in col_list:
        col_qs[col] = q1, q3 = df[col].quantile([0.25, 0.75])
    for col in col_list:
        iqr = col_qs[col][0.75] - col_qs[col][0.25]
        lower_fence = col_qs[col][0.25] - (k*iqr)
        upper_fence = col_qs[col][0.75] + (k*iqr)
        df = df[(df[col] > lower_fence) & (df[col] < upper_fence)]
    return df  

In [41]:
remove_outliers(df, df.select_dtypes(exclude='object').drop(columns=cols).columns.to_list(), 1.5)

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,propertylandusedesc


In [42]:
df.isna().sum()

id                                 0
parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                135
calculatedfinishedsquarefeet      81
finishedsquarefeet12             246
fips                               0
fullbathcnt                      135
latitude                           0
longitude                          0
lotsizesquarefeet                366
propertycountylandusecode          0
propertylandusetypeid              0
rawcensustractandblock             0
regionidcity                    1036
regionidcounty                     0
regionidzip                       26
roomcnt                            0
yearbuilt                        114
structuretaxvaluedollarcnt        82
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          4
censustractandblock              121
logerror                           0
t

In [43]:
df.columns

Index(['id', 'parcelid', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'logerror', 'transactiondate',
       'propertylandusedesc'],
      dtype='object')

In [44]:
mean_cols = ['calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'lotsizesquarefeet', 
             'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount']
mode_cols = ['calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'censustractandblock', 'yearbuilt']

In [45]:
for col in mean_cols:
    df[col].fillna(df[col].mean(), inplace=True)
for col in mode_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


In [46]:
df.isna().sum()

id                              0
parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
propertylandusetypeid           0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
logerror                        0
transactiondate                 0
propertylandusedesc             0
dtype: int64